In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import *
import matplotlib
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.collections import PatchCollection
import datetime
import PySimpleGUI as sg

# define fonts for the plots
font = {'family': 'Arial',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

Downloading and setting up COVID Data DFs

In [2]:
#download most recent data from the cdc website
url = 'https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD'
d = pd.read_csv(url,parse_dates = ['submission_date'])
j = np.datetime64("2020-02-29")
fd = d['submission_date']> j
data = d[fd]
data.loc[:,'submission_date'] = data['submission_date'].dt.strftime('%Y-%m-%d')

#combining NY and NYC data for the map
mapdata = pd.read_csv(url)
mapdata.loc[mapdata['state'] == 'NYC', 'state'] = 'NY'

#creating copy for the last seven days
lastWeekdata = mapdata.copy()


C:\Users\nilsr\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [3]:
# create dict with the state names and abrv since state name not in data
stateDict = {'US':'United States','AK':'Alaska', 'AL':'Alabama', 'AR':'Arkansas', 'AS':'American Samoa', 'AZ':'Arizona', 'CA':'California', 'CO':'Colorado', 'CT':'Connecticut', 'DC':'District of Columbia', 'DE':'Delaware', 'FL':'Florida', 'FSM':'Federated States of Micronesia', 'GA':'Georgia', 'GU':'Guam', 'HI':'Hawaii', 'IA':'Iowa', 'ID':'Idaho', 'IL':'Illinois', 'IN':'Indiana', 'KS':'Kansas', 'KY':'Kentucky', 'LA':'Louisiana', 'MA':'Massachusetts', 'MD':'Maryland', 'ME':'Maine', 'MI':'Michigan', 'MN':'Minnesota', 'MO':'Missouri', 'MP':'Northern Mariana Islands', 'MS':'Mississippi', 'MT':'Montana', 'NC':'North Carolina', 'ND':'North Dakota', 'NE':'Nebraska', 'NH':'New Hampshire', 'NJ':'New Jersey', 'NM':'New Mexico', 'NV':'Nevada', 'NY':'New York (excludes NYC)', 'NYC':'New York City', 'OH':'Ohio', 'OK':'Oklahoma', 'OR':'Oregon', 'PA':'Pennsylvania', 'PR':'Puerto Rico', 'PW':'Palau', 'RI':'Rhode Island', 'RMI':'Republic of Marshall Islands', 'SC':'South Carolina', 'SD':'South Dakota', 'TN':'Tennessee', 'TX':'Texas', 'UT':'Utah', 'VA':'Virginia', 'VI':'Virgin Islands', 'VT':'Vermont', 'WA':'Washington', 'WI':'Wisconsin', 'WV':'West Virginia', 'WY':'Wyoming'}
namesDF = pd.DataFrame(list(stateDict.items()), columns = ['abv','name'])

#Create a list of all the state names
stateNames = namesDF['name'].values.tolist()

#create list pf names for map to work with
mapNamesDF = pd.DataFrame(list(stateDict.items()), columns = ['abv','name'])
mapNamesDF['pop'] = np.zeros(len(stateNames))


In [4]:
#Create a list of all the state abreviations
stateAbrvs = sorted(pd.DataFrame(data, columns=['state']).drop_duplicates().values.tolist())
stateAbrvs = [ item for elem in stateAbrvs for item in elem]

#Returns state abreviation based on the state name
def stateAbrv(state):
    indx =  namesDF.loc[namesDF['name']==state].index[0]
    abrv = namesDF['abv'][indx]
    return abrv

#returns state name from abreviation
def stateN(abrv):
    indx =  namesDF.loc[namesDF['abv']==abrv].index[0]
    state = namesDF['name'][indx]
    if abrv == 'NY':
        state = 'New York'
    return state


Downloading and formatting the Population Data

In [5]:
#Create DF with the population data from the census burrow
popUrl = 'http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-popchg2010_2019.csv'
popDF = pd.read_csv(popUrl)

#create list with the states that have population information delete the ones that don't
popNames = popDF['NAME'].values.tolist()
mapNames = mapNamesDF['name'].values.tolist()
for st in stateNames:
    if st in ['New York City','American Samoa','Federated States of Micronesia','Guam','Northern Mariana Islands','Palau','Republic of Marshall Islands','Virgin Islands'] and st in mapNames:
        indx =  mapNamesDF.loc[mapNamesDF['name']==st].index[0]
        mapNamesDF.drop([indx], inplace = True)
    if st in "New York (excludes NYC)" and st in mapNames:
        mapNamesDF.loc[mapNamesDF['name'] == st, 'name'] = 'New York'
# add the population data to the namesDF for all the states that have info
for st in mapNamesDF['name'].values.tolist():
    if st in popNames:
        pop = popDF.loc[popDF['NAME'] == st,'POPESTIMATE2019'].iloc[0]
        mapNamesDF.loc[mapNamesDF['name'] == st, 'pop'] = pop
    

Functions that Return Data for Total Case maps

In [6]:
# create lists with total cases and rate
totalCasesDF = mapdata.groupby('state')['new_case'].sum()

def totalCases():
    totalCasesList = {}
    maxCases = 0
    minCases = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            cases = float(abs(totalCasesDF[item]))
            if cases>maxCases:
                maxCases = cases
            if cases < minCases:
                minCases = cases
            totalCasesList[sta]=cases
    return totalCasesList, maxCases, minCases
x1 = totalCases()

def totalCasesRate():        
    totalCasesRateList = {}
    maxCasesRate = 0
    minCasesRate = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            cases = float(abs(totalCasesDF[item]))
            caseRate = cases/popu*100000
            if caseRate>maxCasesRate:
                maxCasesRate = caseRate
            if caseRate < minCasesRate:
                minCasesRate = caseRate 
            totalCasesRateList[sta] =  caseRate
    return totalCasesRateList, maxCasesRate, minCasesRate
x2 = totalCasesRate()

Functions that Return Data for Total Death maps

In [7]:
# create lists with total deaths and rate
totalDeathsDF = mapdata.groupby('state')['new_death'].sum()

def totalDeaths():
    totalDeathList = {}
    maxDeath = 0
    minDeath = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            deaths = abs(float(totalDeathsDF[item]))
            if deaths>maxDeath:
                 maxDeath = deaths 
            if deaths < minDeath:
                minDeath = deaths 
            totalDeathList[sta] =  deaths
    return totalDeathList, maxDeath, minDeath
x3 = totalDeaths()

def totalDeathRate():
    totalDeathRateList = {}
    maxDeathRate = 0
    minDeathRate = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            deaths = abs(float(totalDeathsDF[item]))
            deathRate = deaths/popu*100000
            if deathRate>maxDeathRate:
                maxDeathRate = deathRate
            if deathRate < minDeathRate:
                minDeathRate = deathRate 
            totalDeathRateList[sta] =  deathRate
    return totalDeathRateList, maxDeathRate, minDeathRate
x4 = totalDeathRate()


Extracts the Data from the last 7 Days

In [8]:
#Create a list with all the dates
dates = sorted((data['submission_date']).drop_duplicates().values.tolist())
x_values = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in dates]


# find last 7 days reported for the 7 day view
dateList = pd.to_datetime(data['submission_date'])
lastDate = dateList.max()
dRange = pd.date_range(end =lastDate, periods = 7)
dRange = pd.to_datetime(dRange).astype(str)

# Extracts only the rows from the last 7 days
lastWeekdata['submission_date'] = pd.to_datetime(lastWeekdata['submission_date']).astype(str)
lastWeekdata = lastWeekdata[lastWeekdata.submission_date.isin(dRange)]

Functions that Return Data for Last 7 Days Case maps

In [9]:
# create lists with total cases and rate
weekCasesdata = lastWeekdata.groupby('state')['new_case'].sum()

def weekTotalCases():
    weekCasesList = {}
    maxCases = 0
    minCases = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            cases = abs(float(weekCasesdata[item]))
            if cases>maxCases:
                maxCases = cases
            if cases < minCases:
                minCases = cases
            caseRate = cases/popu*100000
            weekCasesList[sta]=cases
    return weekCasesList, maxCases, minCases
x5 = weekTotalCases()
    
def weekTotalCasesRate():
    totalweekCasesRateList = {}
    maxCasesRate = 0
    minCasesRate = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            caseRate = abs(float(weekCasesdata[item]))/popu*100000
            if caseRate>maxCasesRate:
                maxCasesRate = caseRate
            if caseRate < minCasesRate:
                minCasesRate = caseRate 
            totalweekCasesRateList[sta] =  caseRate
    return totalweekCasesRateList, maxCasesRate, minCasesRate
x6 = weekTotalCasesRate()

Functions that Return Data for Last 7 Days Case maps

In [10]:
# create lists with total deaths and rate
weekDeathsdata = lastWeekdata.groupby('state')['new_death'].sum()
def weekTotalDeaths():
    maxDeath = 0
    minDeath = 10000000
    totalDeathList = {}
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            deaths = abs(float(weekDeathsdata[item]))
            if deaths>maxDeath:
                 maxDeath = deaths
            if deaths < minDeath:
                minDeath = deaths 
            totalDeathList[sta] =  deaths
    return totalDeathList, maxDeath, minDeath
x7 = weekTotalDeaths()

def weekTotalDeathRate():
    totalDeathRateList = {}
    maxDeathRate = 0
    minDeathRate = 10000000
    for item in stateAbrvs:
        sta = stateN(item)
        if sta in popNames:
            popu = float(mapNamesDF.loc[mapNamesDF['name'] == sta,'pop'].iloc[0])
            deaths = abs(float(weekDeathsdata[item]))
            deathRate = deaths/popu*100000
            if deathRate>maxDeathRate:
                maxDeathRate = deathRate
            if deathRate < minDeathRate:
                minDeathRate = deathRate 
            totalDeathRateList[sta] =  deathRate
    return totalDeathRateList, maxDeathRate, minDeathRate

x8 = weekTotalDeathRate()

Using the data from above creates the heat maps

In [11]:
def heatMap(vtm,tit,d=''):
    # Lambert Conformal map of lower 48 states.
    rcParams['figure.figsize'] = 20.5,9
    fig = plt.figure()
    mapAx = fig.add_subplot(111)
    title = tit + d 
    mapAx.set_title(title, fontdict=font)
    m = Basemap(llcrnrlon=-119,llcrnrlat=20,urcrnrlon=-64,urcrnrlat=49,
            projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
    # draw state boundaries.
    # data from U.S Census Bureau
    # https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html
    shp_info = m.readshapefile('cb_2019_us_state_500k','states',drawbounds=True)
    # values to map the colors by
    vals = vtm[0]
    
    # choose a color for each state based on population density.
    colors={}


    statenames=[]
    cmap_reversed = cm.get_cmap('viridis_r')
    cmap = plt.cm.viridis # use  colormap
    patches = []
    #sets the upper and lower color range
    vmin = vtm[2]
    if float(vtm[1]) == 0:
        vmax = .001
    else:
        vmax = vtm[1] # set range.
    for shapedict in m.states_info:
        statename = shapedict['NAME']
        # skip DC and Puerto Rico.
        if statename not in ['Puerto Rico','American Samoa','Guam','United States Virgin Islands','Commonwealth of the Northern Mariana Islands']:
            pop = vals[statename]
            # calling colormap with value between 0 and 1 returns
            # rgba value.  Invert color range (hot colors are high
            # population), take sqrt root to spread out colors more.
            colors[statename] = cmap(1.-((abs(pop)-vmin)/(vmax-vmin))**(3/5))[:3]
        statenames.append(statename)
    
    
    # load the shapefile, use the name 'states'
    m.readshapefile('cb_2019_us_state_500k', name='states', drawbounds=True)

    badIslands = [389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400,  385, 386, 387, 388]
    
    # cycle through state names, color each one.
    for nshape,seg in enumerate(m.states):
        # skip DC, Puerto Rico and others not on the map
        if statenames[nshape] not in ['Puerto Rico','American Samoa','Guam','United States Virgin Islands','Commonwealth of the Northern Mariana Islands']:
            color = rgb2hex(colors[statenames[nshape]]) 
            if statenames[nshape]  in ['Alaska', 'Hawaii','District of Columbia']:
                if statenames[nshape] == 'Hawaii' and nshape not in badIslands:
                    seg = list((b[0] + 5300000, b[1]-1550000) for b in seg)
                if statenames[nshape] == 'District of Columbia':
                    seg = list((b[0]*8-4070000*7, b[1]*8-1950000*7 ) for b in seg)
                # Alaska is large. Rescale it.
                elif statenames[nshape] == 'Alaska':
                    seg = list((b[0]*.35 + 1100000, b[1]*.35 - 1300000) for b in seg)
            poly = Polygon(seg,seg,facecolor=color,edgecolor='black',linewidth=1)
            mapAx.add_patch(poly)
            patches.append(poly)
            
    # adds arrows and labels for DC, Alaska and Hawaii
    plt.annotate('D.C', xy=(4595000, 1841000),  xycoords='data',
                    xytext=(4600000, 2050000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    plt.annotate('Alaska', xy=(970000, 600000),  xycoords='data',
                    xytext=(753000, 233000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    plt.annotate('Hawaii', xy=(1730000, 222000),  xycoords='data',
                    xytext=(2000000, 200000), textcoords='data',
                    arrowprops=dict(arrowstyle="->"),fontsize = 16)
    
    # add a colorbar to the map as a legend
    p = PatchCollection(patches, cmap=cmap_reversed)
    p.set_array([])
    cb = fig.colorbar(p, pad = .01,ticks=[ 0.01,.5, .99] )
    cb.ax.set_yticklabels(['Lowest', 'Medium', 'Highest'],fontsize = 16)
    fig.patch.set_facecolor('white')
    return fig

Creating all the maps before the GUI loads so that changing between them is faster

In [12]:
mapfig1 = heatMap(x1, 'Total Number of COVID Cases by State Since January 2020')
plt.close()
mapfig2 = heatMap(x2, 'Rate of COVID Cases per 100,000 People by State Since January 2020')
plt.close()
mapfig3 = heatMap(x3, 'Total Number of COVID Deaths by State Since January 2020')
plt.close()
mapfig4 = heatMap(x4, 'Rate of COVID Deaths per 100,000 People by State Since January 2020')
plt.close()
lastDate = str(lastDate)
mapfig5 = heatMap(x5, 'Total Number of COVID Cases by State for the Last 7 Days',lastDate)
plt.close()
mapfig6 = heatMap(x6, 'Rate of COVID Cases per 100,000 People by State for the Last 7 Days',lastDate)
plt.close()
mapfig7 = heatMap(x7, 'Total Number of COVID Deaths by State for the Last 7 Days',lastDate)
plt.close()
mapfig8 = heatMap(x8, 'Rate of COVID Deaths per 100,000 People by State for the Last 7 Days',lastDate)
plt.close()

Fuction that creates all the bar plots for the first tab

In [13]:
def dataByDate(state,metric):
    if state =='United States' or state == '':
        plotData = data.groupby('submission_date')[metric].sum().values.tolist()
    else:
        abrv = stateAbrv(state)
        casesForAStateDF = data.loc[data['state'] == abrv]
        plotData = casesForAStateDF.groupby(['submission_date'])[metric].sum()
        plotData[plotData<0]=0
    rcParams['figure.figsize'] = 20.5,9
    fig, ax = plt.subplots()
    ax = plt.gca()
    formatter = matplotlib.dates.DateFormatter("%Y-%m-%d")
    ax.xaxis.set_major_formatter(formatter)
    locator = matplotlib.dates.DayLocator()
    ax.xaxis.set_major_locator(locator)
    ax.bar(x_values,plotData,color = 'green',edgecolor = 'white',width = 1 )
    plt.xticks(x_values[::40])
    if metric == 'new_case':
        title = str('Daily Trend of New COVID cases in ' + state)
        plt.ylabel('Cases per Day')
    if metric == 'tot_cases':
        title = str('Trend of Total New COVID Cases in ' + state)
        plt.ylabel('Total Cases')
    if metric == 'new_death':
        title = str('Daily Trend of New COVID Deaths in ' + state)
        plt.ylabel('Deaths per Day')
    if metric == 'tot_death':
        title = str('Trend of Total COVID Deaths in ' + state)
        plt.ylabel('Total Deaths')
    plt.title(title, fontdict=font)
    
    plt.xlabel('Date', fontdict=font)
    return fig

Creates the GUI using PySimpleGUI and adds the maps and charts 

In [14]:
matplotlib.use('TkAgg')

# ------------------------------- Beginning of Matplotlib helper code -----------------------

def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=.1)
    return figure_canvas_agg

# ------------------------------- Beginning of GUI CODE -------------------------------
sg.theme('green')
# define the window layout
# tab 1 layout defined
tab1_col1 = [[sg.Text('Select a region to visualize')],
             [sg.Combo(values=stateNames, size=(20, 6),enable_events=True, key='combo')]]
tab1_col2 = [[sg.Text('View:')],[sg.Text('')]]
tab1_col3 = [[sg.Radio('Cases', "RadioView", key='case', default=True,enable_events=True)],
             [sg.Radio('Deaths', "RadioView", key='death',enable_events=True)]]
tab1_col4 =[[sg.Text('Metric:')],[sg.Text('')]]
tab1_col5 =[[sg.Radio('Daily Trends', "RadioMetric", key='daily', default=True,enable_events=True)],
             [sg.Radio('Total', "RadioMetric", key='total',enable_events=True)]]
tab1_layout = [[sg.Column(tab1_col1),sg.Column(tab1_col2),sg.Column(tab1_col3),sg.Column(tab1_col4),sg.Column(tab1_col5)],
               [sg.Canvas(key='-CANVAS-')]]
              

# tab 2 layout defined
tab2_col1 = [[sg.Text('View:')],[sg.Text('')]]
tab2_col2 = [[sg.Radio('Cases', "RadioView2", key='mapcase', default=True,enable_events=True)],
             [sg.Radio('Deaths', "RadioView2", key='mapdeath',enable_events=True)]]
tab2_col3 = [[sg.Text('Time Period:')],[sg.Text('')]]
tab2_col4 =[[sg.Radio('Last 7 Days', "timeperiod", key='lastweek', default=True,enable_events=True)],
             [sg.Radio('Total', "timeperiod", key='alltime',enable_events=True)]]
tab2_col5 =[[sg.Text('Metric:')],[sg.Text('')]]
tab2_col6 =[[sg.Radio('Count', "RadioMetric2", key='maptotal', default=True,enable_events=True)],
             [sg.Radio('Rate per 100,000', "RadioMetric2", key='rate',enable_events=True)]]
tab2_layout = [[sg.Column(tab2_col1),sg.Column(tab2_col2),sg.Column(tab2_col3),sg.Column(tab2_col4),sg.Column(tab2_col5),sg.Column(tab2_col6)],
               [sg.Canvas(key='-MAPCANVAS-')]]

#combining everything for final layout          
layout = [[sg.TabGroup([[sg.Tab('US and State Trends', tab1_layout,key='tab1'),
                         sg.Tab('Cases and Deaths by State', tab2_layout, key='tab2')]])],
              [sg.Button('Exit')]]

# create the form and show it without the plot
window = sg.Window('COVID Data Visualizer', layout, finalize=True, element_justification='right',font='Arial 12')

# add the default plots to the window
mapfig = mapfig5
pyplot.close()
fig_mapcanvas_agg = draw_figure(window['-MAPCANVAS-'].TKCanvas, mapfig)

fig = dataByDate('','new_case')
pyplot.close()
fig_canvas_agg = draw_figure(window['-CANVAS-'].TKCanvas, fig)

canvas = window['-CANVAS-']
mapcanvas = window['-MAPCANVAS-']

while True:  # Event Loop
    event, values = window.read()
    print(event, values)
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    if event in ['combo','case','death','daily','total']:
        state = values['combo']
        # Update the "canvas" element to be the value of "combo" element
        if values['case']==True and values['daily']==True:
            fig = dataByDate(state,'new_case')
            pyplot.close()
        if values['case']==True and values['total']==True:
            fig = dataByDate(state,'tot_cases')
            pyplot.close()
        if values['death']==True and values['daily']==True:
            fig = dataByDate(state,'new_death')
            pyplot.close()
        if values['death']==True and values['total']==True:
            fig = dataByDate(state,'tot_death')
            pyplot.close()
        #has to destroy the last chart before inputing the new one 
        fig_canvas_agg.get_tk_widget().destroy() 
        fig_canvas_agg = draw_figure(window['-CANVAS-'].TKCanvas, fig)
        
    #depending on the combinations of radio buttons selected displays correct map
    if event in ['mapcase','mapdeath','lastweek','alltime','maptotal','rate']:
        if values['mapcase']==True and values['lastweek']==True and values['maptotal']==True:
            mapfig = mapfig5
            pyplot.close()
        if values['mapcase']==True and values['lastweek']==True and values['rate']==True:
            mapfig = mapfig6
            pyplot.close()
        if values['mapcase']==True and values['alltime']==True and values['maptotal']==True:
            mapfig = mapfig1
            pyplot.close()
        if values['mapcase']==True and values['alltime']==True and values['rate']==True:
            mapfig = mapfig2
            pyplot.close()
        if values['mapdeath']==True and values['lastweek']==True and values['maptotal']==True:
            mapfig = mapfig7
            pyplot.close()
        if values['mapdeath']==True and values['lastweek']==True and values['rate']==True:
            mapfig = mapfig8
            pyplot.close()
        if values['mapdeath']==True and values['alltime']==True and values['maptotal']==True:
            mapfig = mapfig3
            pyplot.close()
        if values['mapdeath']==True and values['alltime']==True and values['rate']==True:
            mapfig = mapfig4
            pyplot.close()
        #has to destroy the last map before inputing the new one   
        fig_mapcanvas_agg.get_tk_widget().destroy() 
        fig_mapcanvas_agg = draw_figure(window['-MAPCANVAS-'].TKCanvas, mapfig)
        
window.close()   

alltime {'combo': '', 'case': True, 'death': False, 'daily': True, 'total': False, '-CANVAS-': None, 'mapcase': True, 'mapdeath': False, 'lastweek': False, 'alltime': True, 'maptotal': True, 'rate': False, '-MAPCANVAS-': None, 0: 'tab2'}
total {'combo': '', 'case': True, 'death': False, 'daily': False, 'total': True, '-CANVAS-': None, 'mapcase': True, 'mapdeath': False, 'lastweek': False, 'alltime': True, 'maptotal': True, 'rate': False, '-MAPCANVAS-': None, 0: 'tab1'}
combo {'combo': 'Arkansas', 'case': True, 'death': False, 'daily': False, 'total': True, '-CANVAS-': None, 'mapcase': True, 'mapdeath': False, 'lastweek': False, 'alltime': True, 'maptotal': True, 'rate': False, '-MAPCANVAS-': None, 0: 'tab1'}
None None
